In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle


In [13]:
features = pd.read_csv("C:/Nini/Capstone/CSV_Files/ravdess_extracted_features_mel_mfcc_chroma_snippets.csv")
features = features.drop(features.columns[0], axis=1)
temp = shuffle(features)
df = np.random.rand(len(temp)) < 0.8
train = temp[df]
test = temp[~df]

In [14]:
X_train = np.array(train.iloc[:,:-1])
y_train = np.array(train.iloc[:,-1])
X_test = np.array(test.iloc[:,:-1])
y_test = np.array(test.iloc[:,-1])

In [15]:
lb = LabelEncoder()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [16]:
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.long)

In [17]:
batch_size = 256
train_loader = DataLoader(TensorDataset(X_train,y_train), batch_size=batch_size,shuffle=True)
test_loader = DataLoader(TensorDataset(X_test,y_test), batch_size=batch_size, shuffle=False)

In [18]:
class CNNModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=8)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=8)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool1 = nn.MaxPool1d(kernel_size=4, padding=2)
        self.dropout1 = nn.Dropout(0.5)
        self.conv3 = nn.Conv1d(128, 128, kernel_size=8)
        self.bn3 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(kernel_size=4, padding=2)
        self.dropout2 = nn.Dropout(0.5)
        self.conv4 = nn.Conv1d(128, 64, kernel_size=3, padding=2)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc1 = nn.Linear(640, 256)
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)

    # def _compute_flattened_size(self, input_size):
    #     with torch.no_grad():
    #         x = torch.randn(1, 1, input_size)  # Create a dummy input
    #         x = self.conv1(x)
    #         x = F.relu(x)
    #         x = self.conv2(x)
    #         x = F.relu(x)
    #         x = self.pool1(x)
    #         x = self.dropout1(x)
    #         x = self.conv3(x)
    #         x = F.relu(x)
    #         x = self.pool2(x)
    #         x = self.dropout2(x)
    #         x = self.conv4(x)
    #         x = F.relu(x)
    #         x = x.view(1, -1)  # Flatten
    #         self._to_linear = x.shape[1]

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout3(x)
        x = self.fc2(x)
        return x

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNNModel(input_size=X_train.shape[1], num_classes=8).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)

In [20]:
print("X_train shape:", X_train.shape)

X_train shape: torch.Size([4150, 1, 153])


In [21]:
epochs = 1500
for epoch in range(epochs):
    model.train()
    running_loss = 0
    total_loss = 0
    total = 0
    correct = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)  # Get predicted class
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)
    train_accuracy = correct / total
    avg_train_loss = running_loss / len(train_loader)
        # total_loss += loss.item()
        # correct += (outputs.argmax(dim=1) == y_batch).sum().item()
    model.eval()  # Set model to evaluation mode
    val_correct = 0
    val_total = 0
    val_loss = 0.0

    with torch.no_grad():  # Disable gradient computation for validation
        for val_inputs, val_labels in test_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)

            val_outputs = model(val_inputs)
            loss = criterion(val_outputs, val_labels)
            val_loss += loss.item()

            _, val_predicted = torch.max(val_outputs, 1)
            val_correct += (val_predicted == val_labels).sum().item()
            val_total += val_labels.size(0)
    
    val_accuracy = val_correct / val_total
    avg_val_loss = val_loss / len(test_loader)
    # accuracy = correct / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
    print("-" * 50)

Epoch [1/1500]
Train Loss: 2.0064, Train Acc: 0.1704
Val Loss: 1.9591, Val Acc: 0.2098
--------------------------------------------------
Epoch [2/1500]
Train Loss: 1.9320, Train Acc: 0.2166
Val Loss: 1.9207, Val Acc: 0.2456
--------------------------------------------------
Epoch [3/1500]
Train Loss: 1.9070, Train Acc: 0.2280
Val Loss: 1.9022, Val Acc: 0.2364
--------------------------------------------------
Epoch [4/1500]
Train Loss: 1.8762, Train Acc: 0.2480
Val Loss: 1.8676, Val Acc: 0.2594
--------------------------------------------------
Epoch [5/1500]
Train Loss: 1.8477, Train Acc: 0.2549
Val Loss: 1.8262, Val Acc: 0.2861
--------------------------------------------------
Epoch [6/1500]
Train Loss: 1.8140, Train Acc: 0.2781
Val Loss: 1.7868, Val Acc: 0.2916
--------------------------------------------------
Epoch [7/1500]
Train Loss: 1.7900, Train Acc: 0.2930
Val Loss: 1.7470, Val Acc: 0.3137
--------------------------------------------------
Epoch [8/1500]
Train Loss: 1.7692,

In [50]:
torch.save(model.state_dict(), "C:/Nini/Capstone/Models/cnn_model_mel_mfcc_chroma_snippets.pth")